In [1]:
# from os import chdir
# from google.colab import drive

# drive.mount("/content/drive", force_remount=True)
# chdir("/content/drive/MyDrive/Eliott/files/")

# !pip install tweet-preprocessor
# %pip install ipython-autotime
# %pip install optuna

ModuleNotFoundError: No module named 'google.colab'

In [2]:
# IMPORTS

import preprocessing as pp
import hyperparameters as hp
import RNN as rnn
import optimising as opt
import json
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier



%load_ext autoreload
%autoreload 2


#
%load_ext autotime


2022-04-15 22:40:20.876281: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-15 22:40:20.876305: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


time: 617 µs (started: 2022-04-15 22:40:24 -04:00)


In [3]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'
X_train, y_train = pp.prepare_dataframe(file_name_train,lemmatising=None, original_class=False)
X_test, y_test = pp.prepare_dataframe(file_name_test,lemmatising=None, original_class=False)

time: 5.75 s (started: 2022-04-15 22:40:24 -04:00)


In [4]:
X_train[0]

'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order'

time: 2.33 ms (started: 2022-04-15 22:40:30 -04:00)


On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys

- 35525 mots sans lemmatisation
- 30794 avec lemmatisation

In [5]:
pipeline_sgd = hp.pipeline_sgd
pipeline_gb = hp.pipeline_gb
pipeline_rf = hp.pipeline_rf
pipeline_lr = hp.pipeline_lr
pipeline_per = hp.pipeline_per
pipeline_svc = hp.pipeline_svc

time: 826 µs (started: 2022-04-15 22:40:32 -04:00)


### Charging the best parameters for training

In [6]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)

data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['Perceptron']
data_svc = data['SVC']

time: 1.69 ms (started: 2022-04-15 22:41:01 -04:00)


### Training on all training data and testing on test data

In [7]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  \n")
    print('------------------------------------------------------\n')
    print('\tClassification Report Train : \n\n', classification_report(y_train, predictions_train, target_names=['Negative','Neutral' ,'Positive']))
    print('------------------------------------------------------\n')
    print('\tClassification Report Test:\n\n',classification_report(y_test,predictions_test, target_names=['Negative', 'Neutral', 'Positive']))
    print('------------------------------------------------------\n')


time: 1.2 ms (started: 2022-04-15 22:41:16 -04:00)


### SGD

In [8]:
data_sgd

{'params': {'clf__alpha': 1.2000000000000002e-05,
  'clf__penalty': 'l1',
  'tfidf__max_df': 0.75,
  'tfidf__min_df': 1,
  'tfidf__ngram_range': [1, 1]},
 'best_score': 0.8769590395125022}

time: 2.67 ms (started: 2022-04-15 22:41:25 -04:00)


In [9]:
model_sgd_best = pipeline_sgd
model_sgd_best.set_params(**(data_sgd["params"]),clf__n_jobs=-1) 

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 5.35 ms (started: 2022-04-15 22:41:32 -04:00)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.75, min_df=1, ngram_range=(1,1))
X_vect = vectorizer.fit_transform(X_train)
X_vect_test = vectorizer.transform(X_test)

time: 609 ms (started: 2022-04-15 22:48:23 -04:00)


In [16]:
from sklearn.linear_model import SGDClassifier

model_sgd = SGDClassifier(alpha=1.2e-5, penalty='l1')

time: 762 µs (started: 2022-04-15 22:48:26 -04:00)


In [17]:
model_sgd.fit(X_vect, y_train)

SGDClassifier(alpha=1.2e-05, penalty='l1')

time: 689 ms (started: 2022-04-15 22:48:26 -04:00)


In [10]:
%%time
"""
Si on a fait la recherche avant, on prend le best_model du gridSearchCv,
Sinon, on reconstruit le modèle avec les best paramètres et on le fit à toutes les données d'entrainement. 
"""

model_sgd_best.fit(X_train,y_train)


CPU times: user 1.33 s, sys: 1.06 s, total: 2.39 s
Wall time: 798 ms


Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 803 ms (started: 2022-04-15 22:41:37 -04:00)


In [11]:
testing(model_sgd_best)

train_accuracy : 0.9314147854557987 
test_accuracy : 0.8724192694547379  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.93      0.93      0.93     15317
     Neutral       0.92      0.89      0.90      7392
    Positive       0.94      0.95      0.94     17912

    accuracy                           0.93     40621
   macro avg       0.93      0.92      0.93     40621
weighted avg       0.93      0.93      0.93     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.88      0.87      0.88      1631
     Neutral       0.85      0.83      0.84       604
    Positive       0.87      0.89      0.88      1543

    accuracy                           0.87      3778
   macro avg       0.87      0.86      0.87      3778
weighted avg       0.87      0.87      0.

In [18]:
def testing_vect(model):
    predictions_train = model.predict(X_vect)
    predictions_test = model.predict(X_vect_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  \n")
    print('------------------------------------------------------\n')
    print('\tClassification Report Train : \n\n', classification_report(y_train, predictions_train, target_names=['Negative','Neutral' ,'Positive']))
    print('------------------------------------------------------\n')
    print('\tClassification Report Test:\n\n',classification_report(y_test,predictions_test, target_names=['Negative', 'Neutral', 'Positive']))
    print('------------------------------------------------------\n')


time: 1.66 ms (started: 2022-04-15 22:48:49 -04:00)


In [19]:
testing_vect(model_sgd)

train_accuracy : 0.9314147854557987 
test_accuracy : 0.8676548438327157  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.93      0.93      0.93     15317
     Neutral       0.92      0.89      0.90      7392
    Positive       0.94      0.95      0.94     17912

    accuracy                           0.93     40621
   macro avg       0.93      0.92      0.93     40621
weighted avg       0.93      0.93      0.93     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.88      0.87      0.87      1631
     Neutral       0.83      0.81      0.82       604
    Positive       0.87      0.89      0.88      1543

    accuracy                           0.87      3778
   macro avg       0.86      0.86      0.86      3778
weighted avg       0.87      0.87      0.